## L’importance de Marseille comme destination des ports du Sud-Est:

Rapporter le nombre de congés délivrés en 1787 (et idem pour  1789) tirés des comptes rendus de la série G5 des ports des amirautés méditerranéennes (voir avec Christine pour extraction du tableau utilisé par Vizsources) avec le nombre d’entrées (dépositions des capitaines de la Santé ET petit cabotage) en provenance (=variable first_point) des ports appartenants aux amirautés des provinces = Corse, Languedoc, Provence, Corse et Roussillon avec nombre d’entrées first_point agrégés par amirauté (departure_admiralty)

des produits exportés depuis Marseille

d'où viennent les marchandises qui sont exportés

- customs_region = Marseille / France
- year = 1789
- export_import = 'Exports'
- best_guess_prodxpartner = '1'
- partner_grouping != 'France'
- origin_province != etranger [list]
- group by origin_province
- sum value

`data/toflit18_all_flows.csv`

In [ ]:
from IPython.display import display
import altair as alt
from altair.utils.data import to_values

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

# 1. Histogrammes d'exports vers l'etranger effectués depuis Marseille / France

In [ ]:
# Renvoyer les colonnes pertinentes dans un grand dataframe (Pyrelation de duckdb) 

import duckdb

foreign_origins = "('Monde', 'Levant et Barbarie', 'Inconnu', 'Asie', 'Italie', 'Amérique', 'Angleterre')"

common_filters = f"""
        WHERE   customs_region IS NOT NULL
        AND year = '1789'
        AND value IS NOT NULL
        AND best_guess_region_prodxpart = '1'
        AND export_import = 'Exports'
        AND partner_grouping != 'France'
        AND origin_province NOT IN {foreign_origins}
"""

csv = '../../data/toflit18_all_flows.csv'
base_relation = duckdb.sql(f"""
SELECT  customs_region,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart,
        origin_province,
        partner_grouping,
FROM read_csv_auto('{csv}', ALL_VARCHAR=TRUE)
{common_filters}
""")

base_relation.show()

In [ ]:
# La fonction pour aggréger les données de Marseille et/ou la France

def aggregate(data, limit):

    aggregated_values_per_origin_province = data.aggregate("""
    origin_province,
    SUM(value) as total_value
    """)
    ordered_values_per_origin_province = duckdb.sql("""
    SELECT *
    FROM aggregated_values_per_origin_province
    ORDER BY total_value DESC
    """)

    if limit:
        values_per_origin_province = ordered_values_per_origin_province.limit(limit)
    else:
        values_per_origin_province = ordered_values_per_origin_province

    values = values_per_origin_province.fetchall()
    columns = values_per_origin_province.columns

    return [ dict(zip(columns, row)) for row in values]


# La fonction pour visualiser les histogrammes d'exports Marseille / France

def visualise(aggregations, title):
    
    VegaLite({
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": title,
        "mark": "bar",
        "data": {
            "values": aggregations
        },
        "encoding": {
            "x": {
                "field": "total_value",
                "type": "quantitative", 
                "title": "value"
            },
            "y": {
                "field": "origin_province",
                "type": "nominal",
                "title": "origin_province",
                "sort": "-x"
            }
        }
    })

## A. Exports effectués depuis Marseille

In [ ]:
# Filtrer le grand dataframe pour Marseille

list_of_regions_values_from_marseille = duckdb.sql(f"""
    FROM base_relation
    WHERE customs_region = 'Marseille'
""")

# Visualiser histogramme d'exports depuis Marseille

title = "La valeur des produits exportés depuis Marseille par province d'origine en 1789"
export_origins_marseille = aggregate(list_of_regions_values_from_marseille, limit=20)

visualise(export_origins_marseille,title)

## B. Exports effectués depuis la France

In [ ]:
# Visualiser histogramme d'exports depuis toute la France

title = "La valeur des produits exportés depuis la France par province d'origine en 1789"
export_origins_france = aggregate(base_relation, limit=20)

visualise(export_origins_france,title)

## C. Comparaison Marseille / France

In [ ]:
# Dictionnaires imbriqués portant sur la valeur des exports effectués depuis toute la France regroupés par origine
export_origins_france = aggregate(base_relation, limit=10)
france_value_per_origin = {i['origin_province']:i['total_value'] for i in export_origins_france}

# Dictionnaires imbriqués portant sur la valeur des exports effectués depuis Marseille regroupés par origine
export_origins_marseille = aggregate(list_of_regions_values_from_marseille, limit=None)
marseille_value_per_origin = {i['origin_province']:i['total_value'] for i in export_origins_marseille}

In [ ]:
ratio_marseille_france = []

for origin, value_france in france_value_per_origin.items():
    if marseille_value_per_origin.get(origin):
        value_marseille = marseille_value_per_origin[origin]
        p = value_marseille/value_france
    else:
        p = 0.0
    ratio_marseille_france.append(
        {'origin_province':origin,
         'total_value':p
        }
    )

VegaLite({
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": "Proportion des exportations en valeur effectuées depuis Marseille en 1789",
        "mark": "bar",
        "data": {
            "values": ratio_marseille_france
        },
        "encoding": {
            "x": {
                "field": "total_value",
                "type": "quantitative", 
                "title": "% de valeur exporté depuis Marseille"
            },
            "y": {
                "field": "origin_province",
                "type": "nominal",
                "title": "origin_province",
                "sort": "-x"
            }
        }
    })

# 2. Par quelles régions transitent les produits exportés à l'etranger du Lyonnais / Languedoc

In [ ]:
common_filters = """
WHERE   customs_region IS NOT NULL
        AND year = '1789'
        AND value IS NOT NULL
        AND best_guess_region_prodxpart = '1'
        AND export_import = 'Exports'
        AND partner_grouping != 'France'
"""

csv = '../../data/toflit18_all_flows.csv'
base_relation = duckdb.sql(f"""
SELECT  customs_region,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart,
        origin_province,
        partner_grouping,
FROM read_csv_auto('{csv}', ALL_VARCHAR=TRUE)
{common_filters}
""")

In [ ]:
# Fonction pour agréger les exports par destination (customs_region)

def aggregate_and_visualise(dataframe, title):

    values_per_customs_region = dataframe.aggregate("""
    customs_region,
    SUM(value) as total_value
    """)
    ordered_values = duckdb.sql("""
    SELECT *
    FROM values_per_customs_region
    ORDER BY total_value DESC
    """)
    
    limited_values = ordered_values.limit(10)

    values = limited_values.fetchall()
    columns = limited_values.columns

    data = [ dict(zip(columns, row)) for row in values]


    VegaLite({
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": title,
        "mark": "bar",
        "data": {
            "values": data
        },
        "encoding": {
            "x": {
                "field": "total_value",
                "type": "quantitative", 
                "title": "total_value"
            },
            "y": {
                "field": "customs_region",
                "type": "nominal",
                "title": "customs_region",
                "sort": "-x"
            }
        }
    })

## A. Export des produits du Lyonnais

In [ ]:
# Filtrer le grand dataframe pour l'origine du Lyonnaise

exports_du_lyonnais = duckdb.sql(f"""
    FROM base_relation
    WHERE origin_province = 'Lyonnais'
""")

# Visualiser histogramme d'exports depuis toute la France

title = "Décomposition des régions de départ des exportations des produits du Lyonnais en 1789"
aggregate_and_visualise(exports_du_lyonnais, title)

## B. Export du produits du Languedoc

In [ ]:
# Filtrer le grand dataframe pour l'origine Languedoc

exports_du_languedoc = duckdb.sql(f"""
    FROM base_relation
    WHERE origin_province = 'Languedoc'
""")

# Visualiser histogramme d'exports depuis toute la France

title = "Décomposition des régions de départ des exportations des produits du Languedoc en 1789"
aggregate_and_visualise(exports_du_languedoc, title)

## C. Export du produits de la Provence

In [ ]:
# Filtrer le grand dataframe pour l'origine de la Provence

exports_du_lyonnais = duckdb.sql(f"""
    FROM base_relation
    WHERE origin_province = 'Provence'
""")

# Visualiser histogramme d'exports depuis toute la France

title = "Décomposition des régions de départ des exportations des produits de la Provence en 1789"
aggregate_and_visualise(exports_du_lyonnais, title)

## D. Export du produits de la Provence

In [ ]:
# Filtrer le grand dataframe pour l'origine de l'Île de France

exports_du_lyonnais = duckdb.sql(f"""
    FROM base_relation
    WHERE origin_province = 'Île-de-France'
""")

# Visualiser histogramme d'exports depuis toute la France

title = "Décomposition des régions de départ des exportations des produits de l'Île-de-France en 1789"
aggregate_and_visualise(exports_du_lyonnais, title)